In [1]:
%pip install --upgrade tensorflow-probability

Requirement already up-to-date: tensorflow-probability in /usr/local/lib/python3.6/dist-packages (0.9.0)
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.1.0


In [9]:
# first, create a TensorFlow constant
const = tf.constant(2.0, name="const")
    
# create TensorFlow variables
b = tf.Variable(20.0, name='b')
c = tf.Variable(2.0, name='c')

In [10]:
print(const)
print(b, c)

Tensor("const:0", shape=(), dtype=float32)
<tf.Variable 'b:0' shape=() dtype=float32_ref> <tf.Variable 'c:0' shape=() dtype=float32_ref>


In [11]:
a = b + c
exp = a**const

In [12]:
print(a, exp)

Tensor("add:0", shape=(), dtype=float32) Tensor("pow:0", shape=(), dtype=float32)


In [13]:
placeholder = tf.placeholder(tf.float32, [1], name='ph')
d = a + placeholder
optim = tf.train.AdamOptimizer(learning_rate=0.001).minimize(exp)

In [14]:
print(placeholder, d)

Tensor("ph:0", shape=(1,), dtype=float32) Tensor("add_1:0", shape=(1,), dtype=float32)


In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
print(sess.graph)
print(sess.list_devices())

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 5369687168762686259), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16139531775510961577), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 6954137712300096333), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:1, XLA_GPU, 17179869184, 8174769956240519845), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15704945460, 2485618092230880277), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:1, GPU, 15704945460, 888315594435644344)]


In [17]:
print(sess.run(b))
print(sess.run(c))
print(sess.run(a))
print(sess.run(exp))

20.0
2.0
22.0
484.0


In [18]:
sess.run(d, feed_dict={placeholder:[30]})

array([52.], dtype=float32)

In [19]:
x = tf.Variable(5.0)
out = 5 + (x-2)**2
optim = tf.train.AdamOptimizer(learning_rate=0.1).minimize(out)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(500):
    _, x_val, out_val = sess.run([optim, x, out])
    if i % 20 == 0:
        print(x_val, out_val)

5.0 14.0
3.1193626 6.2529726
1.9923003 5.000059
1.8476182 5.02322
1.9972924 5.000007
2.0193443 5.0003743
1.9970169 5.000009
1.998391 5.0000024
2.0008497 5.000001
1.9998844 5.0
1.9999473 5.0
2.0000374 5.0
1.9999881 5.0
2.0000017 5.0
2.0000005 5.0
1.9999996 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0
2.0 5.0


In [28]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


# Python optimisation variables
learning_rate = 0.01
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

# now declare the weights connecting the input to the hidden layer
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

# calculate the output of the hidden layer
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                         + (1 - y) * tf.log(1 - y_clipped), axis=1))

optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optim, cross_entropy], 
                            feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

In [3]:
class Test(tf.keras.Model):
    def __init__(self, num_classes, samples_per_class):
        super(Test, self).__init__()
        self.dense = tf.keras.layers.Dense(128, 2)
        self.relu = tf.nn.ReLU()

    def call(self, input_tensor):
        return self.relu(self.dense(input_tensor))

In [4]:
import numpy as np

x = tf.placeholder(tf.float32, [1])
out = 5 + (x-2)**2
grad = tf.gradients(out, x)
xnew = x - 0.01 * grad[0]
# optim = tf.train.AdamOptimizer(learning_rate=0.1).minimize(out)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
xlast = np.array([5.0])
for i in range(500):
    nx, x_val, out_val = sess.run([xnew, x, out], feed_dict={x:xlast})
    xlast = nx
    if i % 20 == 0:
        print(nx , x_val, out_val)

[4.94] [5.] [14.]
[3.9627678] [4.0028243] [9.011305]
[3.3103592] [3.3371012] [6.78784]
[2.8748062] [2.8926594] [5.7968407]
[2.5840275] [2.5959466] [5.355152]
[2.3899012] [2.3978584] [5.1582913]
[2.260301] [2.2656133] [5.0705504]
[2.173779] [2.1773255] [5.0314445]
[2.1160166] [2.1183844] [5.0140147]
[2.0774534] [2.079034] [5.0062466]
[2.0517082] [2.0527635] [5.002784]
[2.0345204] [2.035225] [5.0012407]
[2.023046] [2.0235164] [5.000553]
[2.0153854] [2.0156994] [5.0002465]
[2.0102715] [2.010481] [5.0001097]
[2.0068574] [2.0069973] [5.000049]
[2.004578] [2.0046716] [5.000022]
[2.0030568] [2.0031192] [5.0000095]
[2.0020409] [2.0020826] [5.0000043]
[2.0013623] [2.0013902] [5.000002]
[2.0009098] [2.0009284] [5.000001]
[2.0006075] [2.00062] [5.0000005]
[2.0004056] [2.000414] [5.]
[2.0002706] [2.000276] [5.]
[2.0001807] [2.0001843] [5.]


In [5]:
import tensorflow_probability as tfp

In [6]:
tfd = tfp.distributions
normal = tfd.Normal(loc=0, scale=1)
normal.reparameterization_type

<Reparameteriation Type: FULLY_REPARAMETERIZED>

In [7]:
normal.sample(10)

<tf.Tensor 'Normal_1/sample/Reshape:0' shape=(10,) dtype=float32>